
# AWSDX - Single Server > sdx-continuous-development  
#### (derived from) Functional Test 3.2.1 - Local bridge with Shared NICs

This Jupyter notebook will allow you to create VMs on different sites and worker nodes consistent with requirements for test 3.2.1 for testing Shared NIC with a local bridge.

## Step 1:  Configure the Environment

Before running this notebook, you will need to configure your environment using the [Configure Environment](../../fablib_api/configure_environment/configure_environment.ipynb) notebook. Please stop here, open and run that notebook, then return to this notebook.

**This only needs to be done once.**

## Step 2: Import the FABlib Library


In [12]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

# Update this line to specify your project id
project_id = "7a4f766f-c8f1-4999-a4df-d53a8930265e"

# Uncomment the line below if using 'FABRIC Tutorials' Project
#project_id="a7818636-1fa1-4e77-bb03-d171598b0862"

fablib = fablib_manager(project_id=project_id)

fablib.show_config()

User: yxin@email.unc.edu bastion key is valid!
Configuration is valid


Orchestrator,orchestrator.fabric-testbed.net
Credential Manager,cm.fabric-testbed.net
Core API,uis.fabric-testbed.net
Artifact Manager,artifacts.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,7a4f766f-c8f1-4999-a4df-d53a8930265e
Bastion Host,bastion.fabric-testbed.net
Bastion Username,yxin_0008060646
Bastion Private Key File,/home/fabric/work/fabric_config/fabric_bastion_key
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key


Orchestrator,orchestrator.fabric-testbed.net
Credential Manager,cm.fabric-testbed.net
Core API,uis.fabric-testbed.net
Artifact Manager,artifacts.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,7a4f766f-c8f1-4999-a4df-d53a8930265e
Bastion Host,bastion.fabric-testbed.net
Bastion Username,yxin_0008060646
Bastion Private Key File,/home/fabric/work/fabric_config/fabric_bastion_key
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key


## Step 3: Check your existing slices

Since testing can get confusing, check what slices you actually have. It may print nothing if you have no active slices.

In [2]:
try:
    for slice in fablib.get_slices():
        print(f"{slice}")
except Exception as e:
    print(f"Exception: {e}")

-----------  ------------------------------------
Slice Name   AWSDX sdx-continuous-development
Slice ID     78e07227-18ad-4b6c-9e11-fc7b3d212301
Slice State  StableOK
Lease End    2025-12-18 02:05:23 +0000
-----------  ------------------------------------


## Step 4: Create the Slice

The following creates two nodes with a shared NIC each. This should be run all worker nodes regardless of type.

Two nodes with one NIC component each are created on different workers.  This example uses components of model `NIC_Basic` which are SR-IOV Virtual Function on a 100 Gpbs Mellanox ConnectX-6 PCI device. The VF is accessed by the node via PCI passthrough. 

**Be sure to try different combinations of workers**

In [13]:
from datetime import datetime
from dateutil import tz

name1='Node1'
nic1_name='SharedNIC1'

site='FIU'
###site='DALL'

# since all workers have a standard naming scheme, you can just change the worker
# to move from worker to worker
#worker1=f'{site.lower()}-w1.fabric-testbed.net'
#worker2=f'{site.lower()}-w2.fabric-testbed.net'

cores=12
ram=36
disk=50

#slice_name=f"AWSDX sdx-continuous-development {site} {datetime.now().strftime('%Y-%m-%d-%H-%M')}"
slice_name = "AWSDX sdx-continuous-development"

### List of the VM images

Available images on FABRIC Testbed

```
+------------------------------+
| Name                         |
+------------------------------+
| default_debian_11            |
| default_debian_12            |
| default_rocky_8              |
| default_rocky_9              |
| default_ubuntu_20            |
| default_ubuntu_22            |
| default_ubuntu_24            |
| docker_rocky_8               |
| docker_rocky_9               |
| docker_ubuntu_20             |
| docker_ubuntu_22             |
| docker_ubuntu_24             |
+------------------------------+
```

In [4]:
try:
    #Create Slice
    print(f'Creating slice {slice_name}')
    slice = fablib.new_slice(name=slice_name)

    # Node1
    node1 = slice.add_node(name=name1, site=site, cores=cores, ram=ram, disk=disk, image="docker_rocky_8")
    iface1 = node1.add_component(model='NIC_Basic', name=nic1_name).get_interfaces()[0]
 
    #Submit Slice Request
    slice.submit()
except Exception as e:
    print(f"Exception: {e}")


Retry: 8, Time: 201 sec


ID,78e07227-18ad-4b6c-9e11-fc7b3d212301
Name,AWSDX sdx-continuous-development
Lease Expiration (UTC),2025-12-05 02:05:23 +0000
Lease Start (UTC),2025-12-04 02:05:23 +0000
Project ID,7a4f766f-c8f1-4999-a4df-d53a8930265e
State,StableOK
Email,yxin@email.unc.edu
UserId,4628c0be-4d16-42b4-8dc7-716ca422895a


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
934eb28b-3a5d-4809-aea3-882278c40385,Node1,12,64,100,docker_rocky_8,qcow2,fiu-w2.fabric-testbed.net,FIU,rocky,131.94.57.34,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@131.94.57.34,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


None

Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node,Switch Port
Node1-SharedNIC1-p1,p1,Node1,None,100,config,,06:4A:13:DF:DF:12,enp7s0,enp7s0,None,6,None



Time to print interfaces 202 seconds


## Step 5: Observe the Slice's Attributes

### Print the slice 

In [3]:
try:
    slice = fablib.get_slice(name=slice_name)
    print(f"{slice}")
except Exception as e:
    print(f"Exception: {e}")

Exception: name 'slice_name' is not defined


## Print the Node List

In [3]:
try:
    slice = fablib.get_slice(name=slice_name)

    print(f"{slice.list_nodes()}")
except Exception as e:
    print(f"Exception: {e}")

ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
934eb28b-3a5d-4809-aea3-882278c40385,Node1,12,64,100,docker_rocky_8,qcow2,fiu-w2.fabric-testbed.net,FIU,rocky,131.94.57.34,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@131.94.57.34,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


## Print the Node Details

In [3]:
try:
    slice = fablib.get_slice(name=slice_name)
    for node in slice.get_nodes():
        print(f"{node}")
except Exception as e:
    print(f"Exception: {e}")

-----------------  -----------------------------------------------------------------------------------------------------------------
ID                 934eb28b-3a5d-4809-aea3-882278c40385
Name               Node1
Cores              12
RAM                64
Disk               100
Image              docker_rocky_8
Image Type         qcow2
Host               fiu-w2.fabric-testbed.net
Site               FIU
Management IP      131.94.57.34
Reservation State  Active
Error Message
SSH Command        ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@131.94.57.34
-----------------  -----------------------------------------------------------------------------------------------------------------


## Print the Interfaces

You should see 2 interfaces.

In [4]:
try:
    slice = fablib.get_slice(name=slice_name)
    
    print(f"{slice.list_interfaces()}")
except Exception as e:
    print(f"Exception: {e}")

Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node,Switch Port
Node1-SharedNIC1-p1,p1,Node1,None,100,config,,06:4A:13:DF:DF:12,enp7s0,enp7s0,None,6,None


## Step 6: Configure interfaces, test reachability

##  Configure IP Addresses

### Pick a Subnet

Create a subnet and list of available IP addresses. You can use either IPv4 or IPv6 subnets and addresses.

In [22]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

try:
    subnet = IPv4Network("192.168.1.0/24")
    available_ips = list(subnet)[1:]
except Exception as e:
    print(f"Exception: {e}")

### Configure Node1

Get the node and the interface you wish to configure.  You can use `node.get_interface` to get the interface that is connected to the specified network.  Then `pop` an IP address from the list of available IPs and call `iface.ip_addr_add` to set the IP and subnet.  

Optionally, use the `node.execute()` method to show the results of adding the IP address.

In [17]:
node1 = slice.get_node(name=name1) 
node=node1

In [10]:
try:
    node1 = slice.get_node(name=name1)        
    node1_iface = node1.get_interface(network_name=network_name) 
    node1_addr = available_ips.pop(0)
    node1_iface.ip_addr_add(addr=node1_addr, subnet=subnet)
    
    stdout, stderr = node1.execute(f'ip addr show {node1_iface.get_os_interface()}')
    print (stdout)
    
except Exception as e:
    print(f"Exception: {e}")

Exception: name 'network_name' is not defined


### Select the node 

- node1 : created with the docker_rocky_8 image that already has docker installed. No further action needed for docker installation
- node2 : created with the default_debian_12 image.  Install docker -> https://docs.docker.com/engine/install/debian/


In [ ]:

node = node1

if node == node1:
    print(f'--- Use Node: {node}')
    commands = list()
    commands.append(f'sudo dnf install -y jq')


for command in commands:
    print(f'Executing {command}')
    stdout, stderr = node.execute(command)
    
print('Done')


#### Check Management IP address (IPv4, IPv6)

In [12]:
import ipaddress

def check_ip_version(ip_address):
    try:
        ip_object = ipaddress.ip_address(ip_address)
        if isinstance(ip_object, ipaddress.IPv4Address):
            return "IPv4"
        elif isinstance(ip_object, ipaddress.IPv6Address):
            return "IPv6"
    except ValueError:
        return "Invalid IP address"


In [ ]:
#vi ~/.sense-o-auth.yaml 

### Run end-to-end test



In [18]:

sdx_end_to_end_tests_repo = 'https://github.com/atlanticwave-sdx/sdx-end-to-end-tests.git'
sdx_end_to_end_tests_branch = 'main'
#sdx_end_to_end_tests_branch = "e2e_test_use_cases"
e2e_repo_dir = 'awsdx/sdx-end-to-end-tests'


### 1. Checkout e2e

In [ ]:

commands5 = list()
commands5.append(f'[ ! -d ~/{e2e_repo_dir} ] && mkdir -p ~/{e2e_repo_dir}')
commands5.append(f'cd ~/{e2e_repo_dir} && git init && git remote add origin {sdx_end_to_end_tests_repo}')
commands5.append(f'cd ~/{e2e_repo_dir} && git fetch origin')
commands5.append(f'cd ~/{e2e_repo_dir} && git checkout {sdx_end_to_end_tests_branch}')
commands5.append(f'cd ~/{e2e_repo_dir} && docker compose pull')
# Testing - customize for IPv6

# Add the following block to the docker-compose.yml
# Ref: https://docs.docker.com/reference/compose-file/networks/#the-default-network
#
#networks:
#   default:
#     name: sdxnet
#     enable_ipv6: true
#     ipam:
#       config:
#         - subnet: 2001:db8::/64

#management_ip = node.get_management_ip()

#if check_ip_version(management_ip) == 'IPv6':
#    commands1.append(f'cd ~/{repo_dir} && echo -e "networks:\n  default:\n    name: sdxnet\n    enable_ipv6: true\n    ipam:\n      config:\n        - subnet: 2001:db8::/64" | tee -a docker-compose.yml > /dev/null')


for command in commands5:
    print(f'Executing {command}')
    stdout, stderr = node.execute(command)
    
print('Done')


### 2. Customized image build

### sdx-continuous-development

- https://sdx-docs.readthedocs.io/en/latest/sdx_deploy_single_server.html#


In [19]:

sdx_work_repo = 'https://github.com/atlanticwave-sdx/sdx-continuous-development'
sdx_work_branch = 'main'
repo_dir = 'awsdx/sdx-continuous-development'
docker_compose_dir = repo_dir + '/data-plane'


In [ ]:

commands1 = list()
commands1.append(f'[ ! -d ~/{repo_dir} ] && mkdir -p ~/{repo_dir}')
commands1.append(f'cd ~/{repo_dir} && git init && git remote add origin {sdx_work_repo}')
commands1.append(f'cd ~/{repo_dir} && git fetch origin')
commands1.append(f'cd ~/{repo_dir} && git checkout {sdx_work_branch}')


for command in commands1:
    print(f'Executing {command}')
    stdout, stderr = node.execute(command)
    
print('Done')


In [ ]:
sdx_repo = "sdx-controller"
sdx_branch = "495-failed-to-recover-vlan-when-deleting-l2vpn-after-controller-restarts"
#sdx_branch = "main"
sdx_image = "awsdx/sdx-controller"

commands2=list()

commands2.append(f'pwd')
commands2.append(f'sudo systemctl start docker')
commands2.append(f'cd ~/{docker_compose_dir} && cp template.env .env')
commands2.append(f'cd ~/{docker_compose_dir} && git clone https://github.com/atlanticwave-sdx/{sdx_repo}; cd {sdx_repo}; git checkout {sdx_branch}; docker build -t {sdx_image}:latest .; cd ..;')

for command in commands2:
    print(f'Executing {command}')
    stdout, stderr = node.execute(command)

In [ ]:
sdx_repo = "sdx-lc"
sdx_branch = "194-plugin-client"
#sdx_branch = "main"
sdx_image = "awsdx/sdx-lc"

commands2=list()
commands2.append(f'cd ~/{docker_compose_dir} && git clone https://github.com/atlanticwave-sdx/{sdx_repo}; cd {sdx_repo}; git checkout {sdx_branch}; cp ./env.template ./.env; docker build -t {sdx_image}:latest .; cd ..;')

for command in commands2:
    print(f'Executing {command}')
    stdout, stderr = node.execute(command)
    
print('Done')


### 3. Start Tests

In [20]:

# Testing - start 

commands6 = list()
commands6.append(f'sudo systemctl start docker')
commands6.append(f'cd ~/{e2e_repo_dir} && docker compose up -d  --pull never &> /tmp/docker-compose.log')
commands6.append(f'cd ~/{e2e_repo_dir} && docker compose ps -a')

for command in commands6:
    print(f'Executing {command}')
    stdout, stderr = node.execute(command)
    
print('Done')

Executing sudo systemctl start docker
Executing cd ~/awsdx/sdx-end-to-end-tests && docker compose up -d  --pull never &> /tmp/docker-compose.log
Executing cd ~/awsdx/sdx-end-to-end-tests && docker compose ps -a
NAME                                    IMAGE                         COMMAND                  SERVICE          CREATED          STATUS                    PORTS
sdx-end-to-end-tests-ampath-1           amlight/kytos-sdx:latest      "/bin/bash -x -c '# …"   ampath           14 seconds ago   Up 1 second               6653/tcp, 8181/tcp
sdx-end-to-end-tests-ampath-lc-1        awsdx/sdx-lc:latest           "/bin/bash -x -c '# …"   ampath-lc        14 seconds ago   Up 1 second               
sdx-end-to-end-tests-mininet-1          italovalcy/mininet:latest     "/bin/bash -x -c 'se…"   mininet          14 seconds ago   Up 1 second               6633/tcp, 6640/tcp, 6653/tcp
sdx-end-to-end-tests-mongo-1            mongo:7.0                     "docker-entrypoint.s…"   mongo            14

In [65]:
#docker exec -it sdx-end-to-end-tests-mongo-1  bash
#mongosh --username root_user --password root_pw
#show dbs
#use sdxctldb
#show collections
#show users
#db.links.find()

In [21]:
# Testing - minitnet start

commands7 = list()
commands7.append(f'cd ~/{e2e_repo_dir} && ./wait-mininet-ready.sh')
commands7.append(f'cd ~/{e2e_repo_dir} && ./scripts/run-mininet-interactive.sh ')
for command in commands7:
    print(f'Executing {command}')
    stdout, stderr = node.execute(command)
    
print('Done')

Executing cd ~/awsdx/sdx-end-to-end-tests && ./wait-mininet-ready.sh
Waiting mininet to be ready...ok
Executing cd ~/awsdx/sdx-end-to-end-tests && ./scripts/run-mininet-interactive.sh 
-> starting mininet
-> waiting switches to connect
................................switches connected
Done


In [ ]:
# Testing - provisioning 

#docker compose exec -it mininet curl -s -X POST -H 'Content-type: application/json' http://sdx-controller:8080/SDX-Controller/l2vpn/1.0 -d '{"name": "l2vpn5", "endpoints": [{"port_id": "urn:sdx:port:ampath.net:Ampath3:50", "vlan": "any"}, {"port_id": "urn:sdx:port:tenet.ac.za:Tenet03:50", "vlan": "any"}], "qos_metrics": {"min_bw": {"value": 9}}}' | tee /tmp/l2vpn5

#docker compose exec -it mininet curl -s -X POST -H 'Content-type: application/json' http://sdx-controller:8080/SDX-Controller/l2vpn/1.0 -d '{"name": "l2vpn6", "endpoints": [{"port_id": "urn:sdx:port:ampath.net:Ampath3:50", "vlan": "any"}, {"port_id": "urn:sdx:port:tenet.ac.za:Tenet01:50", "vlan": "any"}], "qos_metrics": {"min_bw": {"value": 9}}}' | tee /tmp/l2vpn6

#docker compose exec -it mininet curl -s http://sdx-controller:8080/SDX-Controller/topology | jq -r '.links[]|.id + " " + (.residual_bandwidth|tostring)'

#./scripts/show-sdx-controller.sh ports

# docker build -t awsdx/sdx-controller:latest .
# Restart sdx-controller
#docker compose up -d sdx-controller --force-recreate --pull never 
# delete the connection
#for ID in $(docker compose exec -it mininet curl -s http://sdx-controller:8080/SDX-Controller/l2vpn/1.0 | jq -r '.[].service_id'); do docker compose exec -it mininet curl -s -X DELETE http://sdx-controller:8080/SDX-Controller/l2vpn/1.0/$ID; done

# docker compose exec -it mininet curl -s -X POST -H 'Content-type: application/json' http://sdx-controller:8080/SDX-Controller/l2vpn/1.0 -d '{"name": "l2vpn-fail", "endpoints": [{"port_id": "urn:sdx:port:ampath.net:Ampath2:50", "vlan": "any"}, {"port_id": "urn:sdx:port:tenet.ac.za:Tenet02:50", "vlan": "any"}], "qos_metrics": {"min_bw": {"value": 9}}}'


In [13]:
# Testing - execute pytest end-to-end test

#commands7.append(f'cd ~/{e2e_repo_dir} && docker compose exec -it mininet python3 -m pytest tests/test_20_use_case_topology.py::TestE2ETopologyUseCases::test_011_intra_domain_link_down_path_found | tee /tmp/docker-compose-exec-pytest.out')
#commands7.append(f'cd ~/{e2e_repo_dir} && docker compose exec -it mininet python3 -m pytest tests/test_20_use_case_topology.py | tee /tmp/docker-compose-exec-pytest.out')
commands7.append(f'cd ~/{e2e_repo_dir} && docker compose exec -it mininet python3 -m pytest | tee /tmp/docker-compose-exec-pytest.out')
commands7.append(f'cd ~/{e2e_repo_dir} && docker compose logs sdx-controller -t > /tmp/docker-compose-logs-sdx-controller.out')


for command in commands7:
    print(f'Executing {command}')
    stdout, stderr = node.execute(command)
    
print('Done')

Executing cd ~/awsdx/sdx-end-to-end-tests && ./wait-mininet-ready.sh
Executing cd ~/awsdx/sdx-end-to-end-tests && docker compose exec -it mininet python3 -m pytest | tee /tmp/docker-compose-exec-pytest.out
============================= test session starts ==============================
platform linux -- Python 3.11.2, pytest-7.2.1, pluggy-1.0.0+repack
rootdir: /sdx-end-to-end-tests
plugins: unordered-0.7.0
collected 95 items

tests/test_01_topology.py .....                                          [  5%]
tests/test_05_l2vpn.py ........                                          [ 13%]
tests/test_06_l2vpn_return_codes.py ..................................   [ 49%]
tests/test_07_l2vpn_return_codes.py .......................              [ 73%]
tests/test_08_l2vpn_return_codes.py ......                               [ 80%]
tests/test_20_use_case_topology.py Xx.x.xx....x                          [ 92%]
tests/test_21_use_case_topology.py x.xx                                  [ 96%]
tests/test

In [11]:
!pip install pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 26.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 43.1 MB/s eta 0:00:00


In [26]:
import os
import time
from urllib.parse import urlparse

import pymongo
db_name = os.environ.get("DB_NAME") or "sdx-controllder-test-db"
mongo_user = os.getenv("MONGO_USER") or "guest"
mongo_pass = os.getenv("MONGO_PASS") or "guest"
mongo_host = os.getenv("MONGO_HOST") or "localhost"
mongo_port = os.getenv("MONGO_PORT") or 27017

mongo_connstring = (
                f"mongodb://{mongo_user}:{mongo_pass}@{mongo_host}:{mongo_port}/"
            )
mongo_client = pymongo.MongoClient(mongo_connstring)

### 4. Stop (Optional)

In [36]:

commands = list()
commands.append(f'cd ~/{e2e_repo_dir} && docker compose down')


for command in commands:
    print(f'Executing {command}')
    stdout, stderr = node.execute(command)
    
print('Done')


Executing cd ~/awsdx/sdx-end-to-end-tests && docker compose down
 Container sdx-end-to-end-tests-mininet-1  Stopping
 Container sdx-end-to-end-tests-sax-lc-1  Stopping
 Container sdx-end-to-end-tests-ampath-lc-1  Stopping
 Container sdx-end-to-end-tests-tenet-lc-1  Stopping
 Container sdx-end-to-end-tests-sdx-controller-1  Stopping
 Container sdx-end-to-end-tests-sdx-controller-1  Stopped
 Container sdx-end-to-end-tests-sdx-controller-1  Removing
 Container sdx-end-to-end-tests-sdx-controller-1  Removed
 Container sdx-end-to-end-tests-ampath-lc-1  Stopped
 Container sdx-end-to-end-tests-ampath-lc-1  Removing
 Container sdx-end-to-end-tests-ampath-lc-1  Removed
 Container sdx-end-to-end-tests-sax-lc-1  Stopped
 Container sdx-end-to-end-tests-sax-lc-1  Removing
 Container sdx-end-to-end-tests-sax-lc-1  Removed
 Container sdx-end-to-end-tests-tenet-lc-1  Stopped
 Container sdx-end-to-end-tests-tenet-lc-1  Removing
 Container sdx-end-to-end-tests-tenet-lc-1  Removed
 Container sdx-end-to-e

### 5. Prune system (Optional)

##### 5.1 Prune docker

In [69]:

commands = list()
commands.append(f'cd ~/{e2e_repo_dir} && docker system prune -a --volumes')


for command in commands:
    print(f'Executing {command}')
    ###stdout, stderr = node.execute(command)
    
print('Done')


Executing cd ~/awsdx/sdx-end-to-end-tests && docker system prune -a --volumes
Done


##### 5.2 Prune directory

In [ ]:

commands = list()
commands.append(f'rm -rf ~/{repo_dir}')


for command in commands:
    print(f'Executing {command}')
    ###stdout, stderr = node.execute(command)
    
print('Done')


### 6. Execute Tests

##### 6.1 Upload scripts

In [22]:
node = node1
local_directory_path = 'sdx-continuous-development'
remote_directory_path = '~/awsdx/'
script_dir = remote_directory_path + local_directory_path
node.upload_directory(local_directory_path=local_directory_path, remote_directory_path=remote_directory_path)


'success'

In [23]:
stdout, stderr = node.execute(f'chmod +x {script_dir}/*')

##### 6.2 Deploy system (This is the same action as Step-7 item 2,3) - Added here for completeness

In [ ]:
###stdout, stderr = node.execute(f'{script_dir}/deploy-single-server.sh deploy {remote_directory_path}')

##### 6.3 Stop the system - Added here for completeness

In [ ]:
###stdout, stderr = node.execute(f'{script_dir}/deploy-single-server.sh stop {remote_directory_path}')

##### 6.4 Start the system - Added here for completeness

In [ ]:
###stdout, stderr = node.execute(f'{script_dir}/deploy-single-server.sh start {remote_directory_path}')

##### 6.5 Query OXPs to make sure all of the expected switches are connected

In [24]:
stdout, stderr = node.execute(f'{script_dir}/deploy-single-server.sh wait_for_oxp_bootup {remote_directory_path}')
stdout, stderr = node.execute(f'{script_dir}/deploy-single-server.sh oxp_query_switches {remote_directory_path}')

bash: /home/rocky/awsdx/sdx-continuous-development/deploy-single-server.sh: No such file or directory
bash: /home/rocky/awsdx/sdx-continuous-development/deploy-single-server.sh: No such file or directory


##### 6.6 Enable the switches and links in OXPs

In [ ]:
stdout, stderr = node.execute(f'{script_dir}/deploy-single-server.sh oxp_switch_enable {remote_directory_path}')

##### 6.7 Query OXPs to make sure all of the expected switches are connected and their details

In [ ]:
stdout, stderr = node.execute(f'{script_dir}/deploy-single-server.sh oxp_query_switches_detail {remote_directory_path}')

##### 6.8 SDX Check topology

In [ ]:
stdout, stderr = node.execute(f'{script_dir}/deploy-single-server.sh sdx_query_topology {remote_directory_path}')

##### 6.9 SDX Check topology links

In [ ]:
stdout, stderr = node.execute(f'{script_dir}/deploy-single-server.sh sdx_query_links {remote_directory_path}')

##### 6.10 Mininet OVS status

In [ ]:
stdout, stderr = node.execute(f'{script_dir}/deploy-single-server.sh mininet_ovs_vsctl_show {remote_directory_path}')

##### 6.11 Check OXPs for services

In [ ]:
stdout, stderr = node.execute(f'{script_dir}/deploy-single-server.sh oxp_query_mef_eline {remote_directory_path}')

##### 6.12 Create L2VPN service 

In [ ]:
stdout, stderr = node.execute(f'{script_dir}/deploy-single-server.sh oxp_create_service_1 {remote_directory_path}')

##### 6.13 Check OXPs for services

In [ ]:
stdout, stderr = node.execute(f'{script_dir}/deploy-single-server.sh oxp_query_mef_eline {remote_directory_path}')

##### 6.14 Create L2VPN service

In [ ]:
stdout, stderr = node.execute(f'{script_dir}/deploy-single-server.sh oxp_create_service_2 {remote_directory_path}')

##### 6.15 Check OXPs for services

In [ ]:
stdout, stderr = node.execute(f'{script_dir}/deploy-single-server.sh oxp_query_mef_eline {remote_directory_path}')

##### 6.16 Test Traffic

In [ ]:
stdout, stderr = node.execute(f'{script_dir}/deploy-single-server.sh test_traffic {remote_directory_path}')

### Docker Hub Login (optional)
Create personal access token -> https://docs.docker.com/security/for-developers/access-tokens/

In [ ]:
# Docker Login - when necessary

docker_access_token = '<DOCKER_PERSONAL_ACCESS_TOKEN>'
docker_user_account = '<DOCKER_USER_ACCOUNT>'

commands = list()
#commands.append(f'echo {docker_access_token} | docker login --username {docker_user_account} --password-stdin')

for command in commands:
    print(f'Executing {command}')
    stdout, stderr = node.execute(command)
    
print('Done')

## Step 8: Delete the Slice

Please delete your slice when you are done with your experiment.

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    ###slice.delete()
except Exception as e:
    print(f"Exception: {e}")